Завдання 1.  Реалізація Rate Limiter з використанням алгоритму Sliding Window для обмеження частоти повідомлень у чаті

In [29]:
import time
import random
from typing import Dict
from collections import deque

# Реалізація класу Sliding Window з Rate Limiter
class SlidingWindowRateLimiter:
    def __init__(self, window_size: int = 10, max_requests: int = 1):
        self.window_size = window_size
        self.max_requests = max_requests
        self.user_windows: Dict[str, deque] = {}

    # Функція очищення застарілих запитів та оновлення активного вікна
    def _cleanup_window(self, user_id: str, current_time: float) -> None:
        if user_id not in self.user_windows:
            return
        window = self.user_windows[user_id]
        while window and current_time - window[0] > self.window_size:
            window.popleft()
        if not window:
            del self.user_windows[user_id]

    # Функція перевірки можливості відправлення повідомлення в поточному вікні
    def can_send_message(self, user_id: str) -> bool:
        current_time = time.time()
        self._cleanup_window(user_id, current_time)
        window = self.user_windows.get(user_id, deque())
        return len(window) < self.max_requests

    # Функція запису нового повідомлення та оновлення історії
    def record_message(self, user_id: str) -> bool:
        if self.can_send_message(user_id):
            current_time = time.time()
            if user_id not in self.user_windows:
                self.user_windows[user_id] = deque()
            self.user_windows[user_id].append(current_time)
            return True
        return False

    # Функція розрахунку часу очікування для відправлення наступного повідомлення
    def time_until_next_allowed(self, user_id: str) -> float:
        current_time = time.time()
        self._cleanup_window(user_id, current_time)
        window = self.user_windows.get(user_id, deque())
        if len(window) < self.max_requests:
            return 0.0
        return max(0.0, self.window_size - (current_time - window[0]))

# Демонстрація роботи
def test_rate_limiter():
    limiter = SlidingWindowRateLimiter(window_size=10, max_requests=1)

    print("\n=== Симуляція потоку повідомлень ===")
    for message_id in range(1, 11):
        user_id = message_id % 5 + 1
        result = limiter.record_message(str(user_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))
        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")
        time.sleep(random.uniform(0.1, 1.0))

    print("\nОчікуємо 4 секунди...")
    time.sleep(4)

    print("\n=== Нова серія повідомлень після очікування ===")
    for message_id in range(11, 21):
        user_id = message_id % 5 + 1
        result = limiter.record_message(str(user_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))
        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")
        time.sleep(random.uniform(0.1, 1.0))

test_rate_limiter()


=== Симуляція потоку повідомлень ===
Повідомлення  1 | Користувач 2 | ✓
Повідомлення  2 | Користувач 3 | ✓
Повідомлення  3 | Користувач 4 | ✓
Повідомлення  4 | Користувач 5 | ✓
Повідомлення  5 | Користувач 1 | ✓
Повідомлення  6 | Користувач 2 | × (очікування 6.3с)
Повідомлення  7 | Користувач 3 | × (очікування 6.3с)
Повідомлення  8 | Користувач 4 | × (очікування 6.8с)
Повідомлення  9 | Користувач 5 | × (очікування 6.7с)
Повідомлення 10 | Користувач 1 | × (очікування 7.0с)

Очікуємо 4 секунди...

=== Нова серія повідомлень після очікування ===
Повідомлення 11 | Користувач 2 | ✓
Повідомлення 12 | Користувач 3 | ✓
Повідомлення 13 | Користувач 4 | ✓
Повідомлення 14 | Користувач 5 | ✓
Повідомлення 15 | Користувач 1 | ✓
Повідомлення 16 | Користувач 2 | × (очікування 6.4с)
Повідомлення 17 | Користувач 3 | × (очікування 6.1с)
Повідомлення 18 | Користувач 4 | × (очікування 6.1с)
Повідомлення 19 | Користувач 5 | × (очікування 6.6с)
Повідомлення 20 | Користувач 1 | × (очікування 7.1с)


Завдання 2. Реалізація Rate Limiter з використанням алгоритму Throttling для обмеження частоти повідомлень у чаті

In [30]:
import time
from typing import Dict
import random

# Реалізація класу ThrottlingRateLimiter
class ThrottlingRateLimiter:
    def __init__(self, min_interval: float = 10.0):
        self.min_interval = min_interval  # мінімальний інтервал між повідомленнями
        self.user_last_message: Dict[str, float] = {}  # час останнього повідомлення для кожного користувача

    # Функція перевірки можливості відправлення повідомлення на основі часу останнього повідомлення
    def can_send_message(self, user_id: str) -> bool:
        current_time = time.time()
        last_time = self.user_last_message.get(user_id)

        # Якщо користувач ще не надсилав повідомлень або минув інтервал — дозволяємо
        if last_time is None or (current_time - last_time) >= self.min_interval:
            return True
        return False

    # Функція запису нового повідомлення з оновленням часу
    def record_message(self, user_id: str) -> bool:
        if self.can_send_message(user_id):
            self.user_last_message[user_id] = time.time()
            return True
        return False

    # Функція розрахунку часу до можливості відправки наступного повідомлення
    def time_until_next_allowed(self, user_id: str) -> float:
        current_time = time.time()
        last_time = self.user_last_message.get(user_id)
        if last_time is None:
            return 0.0
        remaining = self.min_interval - (current_time - last_time)
        return max(0.0, round(remaining, 1))

# Тестування реалізації Throttling
def test_throttling_limiter():
    limiter = ThrottlingRateLimiter(min_interval=10.0)

    print("\n=== Симуляція потоку повідомлень (Throttling) ===")
    for message_id in range(1, 11):
        user_id = message_id % 5 + 1

        result = limiter.record_message(str(user_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))

        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")

        # Випадкова затримка між повідомленнями
        time.sleep(random.uniform(0.1, 1.0))

    print("\nОчікуємо 10 секунд...")
    time.sleep(10)

    print("\n=== Нова серія повідомлень після очікування ===")
    for message_id in range(11, 21):
        user_id = message_id % 5 + 1
        result = limiter.record_message(str(user_id))
        wait_time = limiter.time_until_next_allowed(str(user_id))
        print(f"Повідомлення {message_id:2d} | Користувач {user_id} | "
              f"{'✓' if result else f'× (очікування {wait_time:.1f}с)'}")
        time.sleep(random.uniform(0.1, 1.0))

if __name__ == "__main__":
    test_throttling_limiter()



=== Симуляція потоку повідомлень (Throttling) ===
Повідомлення  1 | Користувач 2 | ✓
Повідомлення  2 | Користувач 3 | ✓
Повідомлення  3 | Користувач 4 | ✓
Повідомлення  4 | Користувач 5 | ✓
Повідомлення  5 | Користувач 1 | ✓
Повідомлення  6 | Користувач 2 | × (очікування 7.8с)
Повідомлення  7 | Користувач 3 | × (очікування 7.8с)
Повідомлення  8 | Користувач 4 | × (очікування 7.4с)
Повідомлення  9 | Користувач 5 | × (очікування 7.3с)
Повідомлення 10 | Користувач 1 | × (очікування 7.7с)

Очікуємо 10 секунд...

=== Нова серія повідомлень після очікування ===
Повідомлення 11 | Користувач 2 | ✓
Повідомлення 12 | Користувач 3 | ✓
Повідомлення 13 | Користувач 4 | ✓
Повідомлення 14 | Користувач 5 | ✓
Повідомлення 15 | Користувач 1 | ✓
Повідомлення 16 | Користувач 2 | × (очікування 7.7с)
Повідомлення 17 | Користувач 3 | × (очікування 7.3с)
Повідомлення 18 | Користувач 4 | × (очікування 7.2с)
Повідомлення 19 | Користувач 5 | × (очікування 7.2с)
Повідомлення 20 | Користувач 1 | × (очікування 6.9